# Project 1: Vacuum Cleaner Agent
## Author: Jared Rand



### Purpose:
Invesitgate the creation, application and assesment of an agent program.

# Introduction:
In this project we were meant to create an agent for a vacuum cleaner that runs in a 10x10 room made of squares. The vacuum cleaner would start in a random position in the room, along with a number of squares that are randomly become dirty based on a given probability. The vacuum runs for a given number of steps each time the program is run and cleans as many dirty spaces as it can reach. The performance of an agent is calculated by dividing the amount of dirt cleaned by the amount dirt total. The measure for each agent is found by running the simulation 200 times for each dirt probability (10%, 20%, 30%) and averaging the resultss.


The agents we were meant to make are seen below:

- Simple (deterministic) reflex agent (just sense, act, sense, act), the agent has a dirt sensor and a bump sensor. It can clean, turn (left or right), and move forward.

- Simple (random) reflex agent, as above but can take a random action.

- A reflex agent which is deterministic but can remember the last 4 sensor inputs and last 4 actions.

# Discussion Of Solution:

## Simple Agent

In [1]:
def program_simple(percepts):
    perceptsstring = ''.join(str(percepts))
    print('Percepts: {}'.format(perceptsstring))

    if 'Dirt' in perceptsstring:
        simple_stats.cleanedcountadd()
        return 'clean'
    if 'Bump' in perceptsstring:
        return 'turnleft'
    simple_stats.movedforwardcountadd()
    return 'moveforward'

For the simple deterministic agent, I had trouble at first figuring out how to get it to do what I wanted. I couldn't figure out why it wasn't changing direction, so I had to change some of the code in the graphic room class to make it actually do what I wanted. It works fine now though so I must have done something right. 

### FROM

In [2]:
if action == 'turnright':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.R)

NameError: name 'action' is not defined

### TO

In [ ]:
if action == 'turnright': # Based on the direction the vacuum is moving, turning right for it is turning left for us, i think.
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            if agent.direction.direction == Direction('down').direction:
                agent.direction = Direction('left')
            elif agent.direction.direction == Direction('left').direction:
                agent.direction = Direction('up')
            elif agent.direction.direction == Direction('up').direction:
                agent.direction = Direction('right')
            elif agent.direction.direction == Direction('right').direction:
                agent.direction = Direction('down')

I'm sure that there's a much better way to do this, but this works just fine. That was the biggest problem I had with this project. Once I fixed that for myself, I was able to figure out how to complete the other two fairly easily. 

## Random Agent

In [ ]:
def program_random(percepts):
    perceptsstring = ''.join(str(percepts))
    print('Percepts: {}'.format(perceptsstring))
    randoms = random.randrange(0, 100)

    if 'Dirt' in perceptsstring:
        random_stats.cleanedcountadd()
        return 'clean'

    if randoms < 10:        ### RANDOM ACTIONS
        return 'turnright'  ### 10% chance to turn right
    if randoms < 25:        ### 15% chance to turn left (has to be between 10 and 25)
        return 'turnleft'   ### 100% to be really cool

    if 'Bump' in perceptsstring:
        return 'turnleft'
    random_stats.movedforwardcountadd()
    return 'moveforward'

The random agent wasn't terribly hard to make. Once I had the simple agent figured out, the only thing different with this one is that it has random components to it. This one has a 10% chance to turn right and a 15% chance to turn right. Besides that, this agent is almost identical to the simple agent.

## Reflex Agent

In [ ]:
def program_remember(percepts):
    perceptsstring = ''.join(str(percepts))
    print('Percepts: {}'.format(perceptsstring))
    remembertable.show()

    if 'Dirt' in perceptsstring:
        remembertable.addsensor('Dirty')
        remembertable.addaction('Vacuum')
        table_stats.cleanedcountadd()
        return 'clean'

    takeaction = remembertable.analyze()    ##based on the table, decides what action should be taken.
    if takeaction is not '':
        return takeaction

    if 'Bump' in perceptsstring:
        remembertable.addsensor('Bump')
        remembertable.addaction('Turn-Left')
        return 'turnleft'

    remembertable.addsensor('Clean')
    remembertable.addaction('Forwards')
    table_stats.movedforwardcountadd()
    return 'moveforward'

This agent required a bit more work to get right. One of the first things I decided to do was make a new class that would keep track of the last 4 actions and sensors the vacuum took. 

In [ ]:
class VacuumTable:
    sensor = ['z', 'z', 'z', 'z']
    actions = ['a', 'a', 'a', 'a']

    def addsensor(self, newsensor):
        sensor = self.sensor
        for y in range(2, -1, -1):
            sensor[y+1] = sensor[y]
        sensor[0] = newsensor

    def addaction(self, newaction):
        actions = self.actions
        for y in range(2, -1, -1):
            actions[y+1] = actions[y]
        actions[0] = newaction

    def show(self):
        print("Sensor Inputs: {}".format(self.sensor))
        print("Actions Taken: {}".format(self.actions))

    def reset(self):
        self.sensor = ['z', 'z', 'z', 'z']
        self.actions = ['a', 'a', 'a', 'a']

    def analyze(self):
        actions = self.actions
        sensors = self.sensor

        if actions.count('Vacuum') > 2:
            print("\n\n Wow! I'm on a roll! :D\n\n")
        if actions.count('Forwards') == 4:   # if it moves forward 4 times in a row, it'll turn left or right
            turnaction = ['turnleft', 'turnright']
            choice = random.choice(turnaction)
            if choice == 'turnleft':
                self.addaction('Turn-Left')
            else:
                self.addaction('Turn-Right')
            return choice   # Turns left or right if it just moved forward 4 times

        if sensors.count('Dirty') > 2:       # if it just cleaned something twice, turn right
            if actions.count('Turn-Left') > 1:
                self.addaction('Turn-Right')
                return 'turnright'
            self.addaction('Turn-Left')
            return 'turnleft'

        return ''

This class lets you keep track of the sensors and actions. When you add more elements into either list, it just shifts everything to the right and overwrites the last one. It also lets the agent analyze whats in each table, so it can decide what actions should be taken, then returns with said action. If nothing works, then it just replies with nothing so the agent knows to continue with its default actions.

# Testing Design and Expected Results:

The way I decided to keep track of the stats for every agent across 200 runs with 100 steps for each run was to make a new class to record everything.

In [ ]:
class KeepTrackOfAllMyThings:
    runnum = 0
    runs = 500
    dirt_count = [0]*runs
    cleaned_count = [0]*runs
    moved_forward_count = [0]*runs

    def dirtcountadd(self, amount):
        self.dirt_count[self.runnum-1] = amount

    def cleanedcountadd(self):
        self.cleaned_count[self.runnum-1] += 1

    def movedforwardcountadd(self):
        self.moved_forward_count[self.runnum-1] += 1

    def average(self):
        length = len(self.dirt_count)
        dirts = 0
        cleans = 0
        moves = 0
        for x in range(0, length):
            dirts += self.dirt_count[x]
            cleans += self.cleaned_count[x]
            moves += self.moved_forward_count[x]
        return dirts, cleans, moves

    def show(self):
        print('\n\n'
              '////////////////////////////\n'
              '//// Agent Stats Round {} ///\n'
              '////////////////////////////\n'
              '/// Spaces Dirty:   {} ///\n'
              '/// Spaces Cleaned: {} ///\n'
              '/// Spaces Moved:   {} ///\n'
              '////////////////////////////\n\n'
              ''.format(self.runnum, self.dirt_count[self.runnum-1], self.cleaned_count[self.runnum-1], self.moved_forward_count[self.runnum-1]))



simple_stats = KeepTrackOfAllMyThings()
random_stats = KeepTrackOfAllMyThings()   ###To keep track of all the stats for each agent
table_stats = KeepTrackOfAllMyThings()

The amount of runs is changed depending on what is given, I just set the default at 500. From there it makes 3 lists of size = number of runs, to keep track of:

- The amount of dirt per run

- The amount of time the vacuum cleaned a dirty square per round

- The amount of times the vacuum moved forward per round

It will also print out the totals and prints out the data measured for each round. 

Personally, I hope the table driven agent performs the best out of the three, just because I like it the most. I do expect it to do the best, but I can also see the random agent doing fairly well. 


# Actual Results:

Below you will find the results for all nine runs. 


### SIMPLE AGENT

<img src= "img/SIMPLE VAC.jpg">

### RANDOM AGENT

<img src= "img/RANDOM VAC.jpg">

### REFLEX AGENT

<img src= "img/TABLE VAC.jpg">

If the images above don't show up for some reason, the results are as follows:

- Simple Agent: --(39.71% for 10% dirt)-- --(39.83% for 20% dirt)-- --(39.72% for 30% dirt)--

- Random Agent: --(54.40% for 10% dirt)-- --(52.38% for 20% dirt)-- --(50.67% for 30% dirt)--

- Table Agent: --(60.96% for 10% dirt)-- --(58.79% for 20% dirt)-- --(59.33% for 30% dirt)--


# Discussion Of Results:

The Table driven agent comes in first with a general performance of 59.69% across all 600 runs. The random agent comes in second with 52.48%, and the simple agent comes in third with 39.75. It's not really surprising that this is the order that they came out in, as the simple agent just moves in a direction until it hits a wall and then just circles the border until the program stops. What was a little surprising is how close the random and table driven agent's results are to each other. I should've expected them to be close seeing as how, while the random agent moves around randomly, the table driven agent only moves around slightly less randomly so it kind of makes sense why they are so similar. Overall, I'm happy the table driven agent performed the best, it's my favorite one.  

# Conclusion:

The table driven agent is the most effcient one out of the three, but the random agent is not very far behind.


# Link To Azure Notebook:

### https://notebooks.azure.com/randja/projects/vacuum-agents


# Code below runs everything:

In [ ]:
###cd pycharmprojects/csc374 && call activate root && jupyter notebook
###cd Dropbox/CSC374 && call activate root && jupyter notebook
### This is just for me, don't look, shhh
from random import randint
from start import *
from KeepTrackOfAllMyThings import *
from start import remembertable

runtimes = 1
iterations = 15   ## Changed these values for the notebook, normally they're 200 and 150 respectively
dirtchance = int(input("Probability for dirt: "))   #######

KeepTrackOfAllMyThings.runs = runtimes

agentchoice = input("Enter Agent Name(Simple, Random, Table): ")    ###### simple_stats, random_stats, table_stats
agentstats = ''
program = ''
progname = ''

if agentchoice.lower() == 'simple':   ####### program_simple, program_random, program_remember
    agentstats = simple_stats
    program = program_simple
    progname = 'program_simple'  #####
elif agentchoice.lower() == 'random':
    agentstats = random_stats
    program = program_random
    progname = 'program_random'
elif agentchoice.lower() == 'table':
    agentstats = table_stats
    program = program_remember
    progname = 'program_remember'
else:
    print("-- Error: Agent Not Recognized --")
    quit()


def runitall():
    room = GraphicRoom(12, 12, color={'Vacuum': (200, 0, 0), 'Dirt': (0, 200, 200), 'Wall': (0, 0, 0), 'Bump': (200, 0, 200)})
    '''the wall is in 0 and location 11'''

    location = [randint(1, 10), randint(1, 10)]
    vac = Vacuum(program)
    room.add_thing(vac, location)
    remembertable.reset()

    def dirtyitup(totaldirt):
        row = 0
        col = 1

        for x in range(100):
            row += 1

            if row > 10:
                row = 1
                col += 1
            if random.randrange(0, 100) < dirtchance:
                room.add_thing(Dirt(), [row, col])
                totaldirt += 1
        return totaldirt

    totaldirt = dirtyitup(0)  # run with 10% chance of dirt
    agentstats.dirtcountadd(totaldirt)
    print(":::::::::::::::::::::::::::::::::::::::::::::::::::::::\n"
          ":::::With a probability of {}%, {} blocks are dirty:::::\n"
          ":::::::::::::::::::::::::::::::::::::::::::::::::::::::".format(dirtchance, totaldirt))

    room.reveal()
    room.run(iterations, 0) #0 Speeds it up so fast it's so nice. Just take it out if you want it to go slow


for x in range(0, runtimes):

    KeepTrackOfAllMyThings.runnum += 1
    runitall()
    agentstats.show()

        # Ignore all these, they look ugly
finalaverages = agentstats.totals()        ###################Time to print out all the results we need
print("\n\n::::::::::::::::::::::::::::::::::::::\n"
      ":::::::::::::: RESULTS :::::::::::::::\n"
      "::::::::::::::::::::::::::::::::::::::\n"
      ":::: Run  {} times :::::::::::::::::::\n"
      ":::: With {} iterations each run :::::\n"
      ":::: And {}% probability for Dirt ::::\n"
      "::::::::::::::::::::::::::::::::::::::\n"
      "".format(runtimes, iterations, dirtchance))
print(":::::::::::::::::::::::::::::::::\n"
      ":::::::::::::  Totals  ::::::::::\n"
      "::: Total Dirty Spaces:   {} ::: \n"
      "::: Total Spaces Cleaned: {} :::\n"
      "::: Total Spaces Moved    {} :::\n"
      ":::::::::::::::::::::::::::::::::\n"
      "".format(finalaverages[0], finalaverages[1], finalaverages[2]))
print("/////////////////////////////////////\n"
      "////////////  Averages  /////////////\n"
      "/// Average Dirty Spaces:   {} /// \n"
      "/// Average Spaces Cleaned: {} ///\n"
      "/// Average Spaces Moved    {} ///\n"
      "/////////////////////////////////////\n"
      "".format((finalaverages[0]/runtimes), (finalaverages[1]/runtimes), (finalaverages[2]/runtimes)))
print("\n!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!\n"
      "!!!!!!! {}'s Average Performance !!!!!\n"
      "!!!!!!!!!!! Cleaned Spaces / Total Dirt !!!!!!!!!!!!\n"
      "!!!!!!!!!!!!!!!!!!!!!!! EQUALS !!!!!!!!!!!!!!!!!!!!!\n"
      "!!!!!!!!!!!!!!!!!!!!!!! {:.2f}% !!!!!!!!!!!!!!!!!!!!!\n"
      "!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!"
      "".format(progname, (finalaverages[1]/finalaverages[0])*100))

# THE RESULTS GET WONKY IN THE NOTEBOOK,
# IT KEEPS TRACK OF THE PREVIOUS RUNS STILL


